# VacationPy by Cenez Tindoc
----

### Description
* The Python code started with using the DF from the WeatherPy by creating a heat map of the using the city data.  A hotel DF was created by selecting the cities that have met the set specifications.  The DF was cleaned out by deleting cities that do not have identified hotel.  A hotel map was later created with a heat map layer.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import gmaps
import requests

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
vacation_df=pd.read_csv("../output_data/cities_df.csv")
vacation_df.head()

,Unnamed: 0,City ID,City,Country,Update Date,Temperature,Cloudiness %,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,5557293,Sitka,US,1580004409,39.31,90,81,57.05,-135.33,44.60,23.04
1,1,3652764,Puerto Ayora,EC,1580004409,77.00,54,37,-0.74,-90.35,77.00,3.00
2,2,4032243,Vaini,TO,1580004409,80.60,90,88,-21.20,-175.20,80.60,12.75
3,3,1295765,Sittwe,MM,1580004410,63.48,0,50,20.15,92.90,63.48,6.60
4,4,1218612,Murgab,TM,1580004314,30.20,75,92,37.50,61.97,30.20,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Store latitude and longitude in locations
glocations = vacation_df[["Latitude", "Longitude"]]
glocations.head()

,Latitude,Longitude
0,57.05,-135.33
1,-0.74,-90.35
2,-21.20,-175.20
3,20.15,92.90
4,37.50,61.97


In [17]:
#convert Humidity as float
humidity = vacation_df["Humidity"].astype(float)
humidity.head()


0    81.0
1    37.0
2    88.0
3    50.0
4    92.0
Name: Humidity, dtype: float64

In [18]:
# Plot map
gmaps.configure(api_key=g_key)

# Plot as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")
fig



Figure(layout=FigureLayout(height='420px'))

In [19]:
# Display figure

# Create a center coordinate pair for our gmap
centerCoord = (24.55,45.55)
# Plot map
fig = gmaps.figure(center=centerCoord,zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(glocations, weights= humidity, dissipating=False, max_intensity=10,point_radius=1)

# Add layer
fig.add_layer(heat_layer)

plt.savefig("../output_data/humid_heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
narrowed_city_df=pd.read_csv("../output_data/cities_df.csv")
#narrowed_city_df=narrowed_city_df(axis = 0, how ='all') 

#Create a Hotel DF that has temp = 70 -80, wind speed =10, humidity <60
narrowed_city_df=narrowed_city_df.loc[narrowed_city_df['Temperature']<80]
narrowed_city_df=narrowed_city_df.loc[narrowed_city_df['Temperature']>70]
narrowed_city_df=narrowed_city_df.loc[narrowed_city_df['Wind Speed']<10]
narrowed_city_df=narrowed_city_df.loc[narrowed_city_df['Humidity']<60]

#Display and save DF
narrowed_city_df.to_csv("../output_data/narrowed_city_df.csv")
len(narrowed_city_df)

72

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#Create a Hotel DF 
hotel_df = narrowed_city_df[['City','Country','Temperature','Latitude','Longitude']]

#Add columns for hotel information
#Used "" for initial entries

hotel_df.head()


,City,Country,Temperature,Latitude,Longitude
1,Puerto Ayora,EC,77.00,-0.74,-90.35
13,Busselton,AU,79.00,-33.65,115.33
47,Hilo,US,75.18,19.73,-155.09
73,Busselton,AU,79.00,-33.65,115.33
83,Hilo,US,75.18,19.73,-155.09


In [22]:
# Define the params dictionary to update each iteration
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify the city where to find the hotel
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    coordinates = f"{lat},{lng}"

        # add location to params dict
    params['location'] = coordinates

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {coordinates}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
        
    try:
        print(f"Closest hotel at {coordinates} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']
        hotel_df.loc[index, 'Rating'] = results[0]['rating']
                       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
              
    print("------E-N-D------")

#https://developers.google.com/places/web-service/supported_types
              

Retrieving Results for Index 1: -0.74,-90.35.
Closest hotel at -0.74,-90.35 is Galapagos Paradise.
------E-N-D------
Retrieving Results for Index 13: -33.65,115.33.
Closest hotel at -33.65,115.33 is Esplanade Hotel.
------E-N-D------
Retrieving Results for Index 47: 19.73,-155.09.
Closest hotel at 19.73,-155.09 is Hilo Hawaiian Hotel.
------E-N-D------
Retrieving Results for Index 73: -33.65,115.33.
Closest hotel at -33.65,115.33 is Esplanade Hotel.
------E-N-D------
Retrieving Results for Index 83: 19.73,-155.09.
Closest hotel at 19.73,-155.09 is Hilo Hawaiian Hotel.
------E-N-D------
Retrieving Results for Index 99: -33.65,115.33.
Closest hotel at -33.65,115.33 is Esplanade Hotel.
------E-N-D------
Retrieving Results for Index 106: 19.73,-155.09.
Closest hotel at 19.73,-155.09 is Hilo Hawaiian Hotel.
------E-N-D------
Retrieving Results for Index 121: -33.65,115.33.
Closest hotel at -33.65,115.33 is Esplanade Hotel.
------E-N-D------
Retrieving Results for Index 134: -33.65,115.33.
C

Closest hotel at -0.74,-90.35 is Galapagos Paradise.
------E-N-D------
Retrieving Results for Index 1291: 18.9,-3.53.
Missing field/result... skipping.
------E-N-D------
Retrieving Results for Index 1314: -33.65,115.33.
Closest hotel at -33.65,115.33 is Esplanade Hotel.
------E-N-D------


In [31]:
hotel_info=hotel_df.dropna()
hotel_info=hotel_info.reset_index(drop=True)
hotel_info.head()

,City,Country,Temperature,Latitude,Longitude,Name,Address,Rating
0,Puerto Ayora,EC,77.00,-0.74,-90.35,Galapagos Paradise,"Avenida Seymour S/N Fragata Esquina, Santa Cruz",3.9
1,Busselton,AU,79.00,-33.65,115.33,Esplanade Hotel,"30-38 Marine Terrace, Busselton",4.0
2,Hilo,US,75.18,19.73,-155.09,Hilo Hawaiian Hotel,"71 Banyan Dr, Hilo",4.1
3,Busselton,AU,79.00,-33.65,115.33,Esplanade Hotel,"30-38 Marine Terrace, Busselton",4.0
4,Hilo,US,75.18,19.73,-155.09,Hilo Hawaiian Hotel,"71 Banyan Dr, Hilo",4.1


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_plt = [info_box_template.format(**row) for index, row in hotel_info.iterrows()]
locations = hotel_info[["Latitude", "Longitude"]]

In [30]:
# Display figure
#Use rating as weights for the heat layer
rating = hotel_info["Rating"]
#Plot as a Hybrid map
hotel_plt = gmaps.figure(map_type="HYBRID")
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights= rating, dissipating=False, max_intensity=10,point_radius=5)
# Add heat layer
hotel_plt.add_layer(heat_layer)
# Display Map
plt.savefig("../output_data/hotel_map.png")
hotel_plt


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>